## <font color='darkblue'>Preface</font>
([article source](https://towardsdatascience.com/beyond-one-hot-17-ways-of-transforming-categorical-features-into-numeric-features-57f54f199ea4)) <font size='3ptx'>**All the encodings that are worth knowing — from OrdinalEncoder to CatBoostEncoder — explained and coded from scratch in Python**</font>
> “Which gradient boostings do you know?”
> “Xgboost, LightGBM, Catboost, HistGradient.”
> “And which categorical encodings do you know?”
> “One-hot.”

![1.png](images/1.png)
<br/>


I wouldn’t be surprised to hear such a conversation during a data science interview. Still, it would be quite striking, since **just a small portion of data-science projects involve machine learning, whereas practically all of them involve some categorical data.**
> **<font color='darkblue'>Categorical encoding</font>** is the process of transforming a categorical column into one (or more) numeric column(s).

**This is necessary because computers are more at ease working with numbers than with strings**. Why is that? Because with numbers it’s easy to find relations (<font color='brown'>such as “bigger”, “smaller”, “double”, “half”</font>). Whereas — when given strings— **a computer can say pretty much only whether they are “equal” or “different”**.

However, despite its impact, <font color='darkred'>**categorical encoding is easily overlooked by data science practitioners.**</font>
> Categorical encoding is a surprisingly underrated topic.

This why I decided to deepen my knowledge of encoding algorithms. I started from a Python library called “[**category_encoders**](https://pypi.org/project/category-encoders/)” (this is the [Github link](https://github.com/scikit-learn-contrib/category_encoders)). Using it is as easy as:
```python
import category_encoders as ce

ce.OrdinalEncoder().fit_transform(x)
```

In [2]:
# !pip install category_encoders

This post is a walkthrough of the 17 encoding algorithms contained in the library . For each algorithm, I provide a short explanation and a Python implementation in few lines of code. The purpose is not to reinvent the wheel, but to realize how the algorithms work under the hood. After all,
> “You don’t understand it, until you can code it”.

## <font color='darkblue'>Not All Encodings Are Created Equal</font>
I have classified the 17 encoding algorithms based on some of their characteristics. And since data scientists love decision trees, let’s make them happy:
![2.png](images/2.png)
<br/>

Here’s what the splits refer to:
* **Supervised/Unsupervised:** when the encoding is based solely on the categorical column, then it’s unsupervised. Otherwise, if the encoding is based on some function of the original column and a second (numeric) column, then it’s supervised.
* **Output dimension:** the encoding of a categorical column may produce one numeric column (output dimension = 1) or many numeric columns (output dimension > 1).
* **Mapping:** if each level has always the same output — whether a scalar (e.g. OrdinalEncoder) or an array (e.g. OneHotEncoder)— then the mapping is unique. On the contrary, if the same level is “allowed” to have different possible outputs, then the mapping is not unique.

## <font color='darkblue'>17 Categorical Encoding Algorithms in 10 Minutes</font>
* <font size='3ptx'>[**1. OrdinalEncoder**](#cate1)</font>
* <font size='3ptx'>[**2. CountEncoder**](#cate2)</font>
* <font size='3ptx'>[**3. OneHotEncoder**](#cate3)</font>
* <font size='3ptx'>[**4. SumEncoder**](#cate4)</font>
* <font size='3ptx'>[**5. BackwardDifferenceEncoder**](#cate5)</font>
* <font size='3ptx'>[**6. HelmertEncoder**](#cate6)</font>
* <font size='3ptx'>[**7. PolynomialEncoder**](#cate7)</font>
* <font size='3ptx'>[**8. BinaryEncoder**](#cate8)</font>
* <font size='3ptx'>[**9. BaseNEncoder**](#cate9)</font>
* <font size='3ptx'>[**10. HashingEncoder**](#cate10)</font>
* <font size='3ptx'>[**11. TargetEncoder**](#cate11)</font>
* <font size='3ptx'>[**12. MEstimateEncoder**](#cate12)</font>
* <font size='3ptx'>[**13. JamesSteinEncoder**](#cate13)</font>
* <font size='3ptx'>[**14. GLMMEncoder**](#cate14)</font>
* <font size='3ptx'>[**15. WOEEncoder**](#cate15)</font>
* <font size='3ptx'>[**16. LeaveOneOutEncoder**](#cate16)</font>
* <font size='3ptx'>[**17. CatBoostEncoder**](#cate17)</font>

<a id='cate1'></a>
### <font color='darkgreen'>1. OrdinalEncoder</font>
Each level is mapped to an integer, from 1 to `L` (<font color='brown'>where `L` is the number of levels</font>). In this case we have used alphabetical order, but any other custom order is acceptable. A simple implementation is as below:
```python
sorted_x = sorted(set(x))
ordinal_encoding = x.replace(dict(zip(sorted_x, range(1, len(sorted_x) + 1))))
```

![3.png](images/3.png)
<br/>

**You may think that ordinal encoding is non-sense, especially if the levels have no intrinsic order**. You are right! In fact, it’s only a representation of convenience, used often to save memory, or as intermediate step for other types of encoding.

<a id='cate2'></a>
### <font color='darkgreen'>2. CountEncoder</font>
Each level is mapped to the number of observations carrying that level. A simple implementation:
```python
count_encoding = x.replace(x.value_counts().to_dict())
```

![4.png](images/4.png)
<br/>

This encoding may be useful as an indicator of the “credibility” of each level. For instance, **a machine learning algorithm may automatically decide to take into account the information brought by the level only its count is above some threshold.**

<a id='cate3'></a>
### <font color='darkgreen'>3. OneHotEncoder</font>
The encoding algorithm for excellence (<font color='brown'>and the most used</font>). Each level is mapped to a dummy column (i.e. a column of 0/1), indicating whether that level is carried by that row. A simple implementation:
```python
one_hot_encoding = ordinal_encoding.apply(
  lambda oe: pd.Series(np.diag(np.ones(len(set(x))))[oe - 1].astype(int))
)
```

![5.png](images/5.png)
<br/>

In [5]:
import numpy as np
'''
np.ones(4) = [1,1,1,1]
np.diag(np.ones(4)) =[
    [1, 0, 0, 0],
    [0, 1, 0, 0], 
    [0, 0, 1, 0],
    [0, 0, 0, 1]
]
'''
np.diag(np.ones(4))[2]

array([0., 0., 1., 0.])

**This implies that, whereas your input is a single column, your output consists of `L` columns** (<font color='brown'>one for each level of the original column</font>). This is why one-hot encoding should be handled with care: you may end up with a dataframe that is far bigger than the original one.

make things understandable at first sight, let’s take one observation for each level. Suppose we have observed a target variable, called `y`, containing the income of each individual (<font color='brown'>in thousands of dollars</font>). Let’s fit a linear regression (<font color='brown'>OLS</font>) on the data.

To make the results easily readable, I have attached the OLS coefficients at the side of the table:
![6.png](images/6.png)
<br/>

In the case of one-hot encoding, the `intercept` has no particular meaning, and the coefficients are added to the intercept to obtain the estimate. In this case, since we have just one observation per level, by adding the `intercept` and the coefficient we obtain the exact value of `y` (<font color='brown'>there is no error</font>).

<a id='cate4'></a>
### <font color='darkgreen'>4. SumEncoder</font>
The implementation code that follows may seem a little obscure at first. But don’t worry: in this case, it’s not so important to understand how the encoding is obtained, but how to use it.
```python
sum_encoding = one_hot_encoding.iloc[:, :-1].apply(
  lambda row: row if row.sum() == 1 else row.replace(0, -1)
  , axis = 1
)
```

**<font color='blue'>SumEncoder</font>** (<font color='brown'>as the next 3 encoders</font>) belongs to a class called **“<font color='darkblue'>contrast encodings</font>”**. These encodings are designed to have a specific behaviour when used in regression problems. In other words, **you use one of these encodings if you want the regression coefficients to have some specific properties.**

In particular, **<font color='blue'>SumEncoder</font> is used when you want the regression coefficients to have zero-sum.** If we take the same data that we have used in the paragraph above and fit a OLS, this is what we get:
![7.png](images/7.png)
<br/>

This time, the intercept corresponds to the mean of `y`. Moreover, by taking `y` of the last level and subtracting it from the `intercept` (`68-50`) we get 18, which is exactly the opposite of the sum of the remaining coefficients (`-15-5+2=-18`). This is precisely the property of sum encoding that I have mentioned above.

<a id='cate5'></a>
### <font color='darkgreen'>5. BackwardDifferenceEncoder</font>
Another contrast encoding (<font color='brown'>like</font> **<font color='blue'>SumEncoder</font>**). **This encoder is useful for ordinal variables, i.e. variables whose levels can be ordered in a meaningful way**. **<font color='blue'>BackwardDifferenceEncoder</font>** is designed to compare adjacent levels:
```python
backward_difference_encoding = ordinal_encoding.apply(
    lambda oe: pd.Series(
        [i / len(set(x)) for i in range(1, oe)] + 
        [- i / len(set(x)) for i in range(len(set(x)) - oe, 0, -1)]
    )
)
```

![8.png](images/8.png)
<br/>

Suppose you have an ordinable variable (<font color='brown'>e.g. education level</font>) and you want to know how it is related to a numeric variable (<font color='brown'>e.g. income</font>). It may be interesting to compare each couple of consecutive levels (<font color='brown'>e.g. bachelors vs. high-school, masters vs. bachelors</font>) with respect to the target variable. This is what **<font color='blue'>BackwardDifferenceEncoder</font>** is designed for. Let’s see an example with the same data from the paragraphs above:
![9.png](images/9.png)
<br/>

The intercept coincides with the mean of `y`. The coefficient of Bachelors is 10, because `y` of Bachelors is 10 higher than High-School, Masters's coefficient equals 7 because `y` of Masters is 7 higher than Bachelors and so on.

<a id='cate6'></a>
### <font color='darkgreen'>6. HelmertEncoder</font>
**<font color='blue'>HelmertEncoder</font>** is very similar to **<font color='blue'>[BackwardDifferenceEncoder](#cate5)</font>**, but instead of being compared just to the previous one, each level is compared with all the previous levels. A simple implementation:
```python
helmert_encoding = ordinal_encoding.apply(
    lambda oe: pd.Series(
        [0] * (oe - 2) + \
        ([oe - 1] if oe > 1 else []) + [-1] * (len(set(x)) - oe)
    )
).div(pd.Series(range(2,len(set(x)) + 1)))
```

![10.png](images/10.png)
<br/>

Let’s see what we would get from a OLS model:
![11.png](images/11.png)
<br/>

PhD’s coefficient is 24, because PhD is 24 higher than the mean of the previous levels `68-((35+45+52)/3)=24`. The same reasoning applies to all the levels.

<a id='cate7'></a>
### <font color='darkgreen'>7. PolynomialEncoder</font>
**Another contrast encoding.**

As its name suggests, PolynomialEncoder is designed to quantify linear, quadratic and cubic behaviors of the target variable with respect to the categorical variable. A simple implementation:
```python
def do_polynomial_encoding(order):
    # code from https://github.com/pydata/patsy/blob/master/patsy/contrasts.py
    n = len(set(x))
    scores = np.arange(n)
    scores = np.asarray(scores, dtype=float)
    scores -= scores.mean()
    raw_poly = scores.reshape((-1, 1)) ** np.arange(n).reshape((1, -1))
    q, r = np.linalg.qr(raw_poly)
    q *= np.sign(np.diag(r))
    q /= np.sqrt(np.sum(q ** 2, axis=1))
    q = q[:, 1:]
    return q[order - 1]

polynomial_encoding = ordinal_encoding.apply(lambda oe: pd.Series(do_polynomial_encoding(oe)))
```

![12.png](images/12.png)
<br/>

I know what you are thinking. How can a numeric variable have a linear (<font color='brown'>or quadratic, or cubic</font>) relation with a variable that is not numeric? This is based on the assumption that the underlying categorical variable has levels that are not only ordinable, but also equally spaced.

For this reason, <font color='darkred'>**I would suggest to use it with care, only when you are sure that the assumption is reasonable.**</font>

<a id='cate8'></a>
### <font color='darkgreen'>8. BinaryEncoder</font>
<font color='blue'>**BinaryEncoder**</font> is basically the same of [**OrdinalEncoder**](#cate1), the only difference is that the integers are converted to binary numbers, then every positional digit is one-hot encoded. A simple implementation:
```python
binary_base = ordinal_encoding.apply(
  lambda oe: str(bin(oe))[2:].zfill(len(bin(len(set(x)))) - 2)
)
binary_encoding = binary_base.apply(lambda bb: pd.Series(list(bb))).astype(int)
```

![13.png](images/13.png)
<br/>

The output consists of dummy columns, as happens for the [**OneHotEncoder**](#cate3), but it leads to a dimensionality reduction with respect to one-hot.

<a id='cate9'></a>
### <font color='darkgreen'>9. BaseNEncoder</font>
<font color='blue'>**BaseNEncoder**</font> is simply a generalization of the [**BinaryEncoder**](#cate8). In fact, in [**BinaryEncoder**](#cate8), the numbers are in base 2, whereas in <font color='blue'>**BaseNEncoder**</font>, numbers are in base `n`, with `n` greater than 1.
```python
def int2base(n, base):
    # return representation of int n in base base
    out = ''
    while n:
        out += str(int(n % base))
        n //= base
    return out[::-1]

base_n = ordinal_encoding.apply(lambda oe: int2base(n = oe, base = base))
base_n_encoding = base_n.apply(
    lambda bn: pd.Series(list(bn.zfill(base_n.apply(len).max())))
).astype(int)
```
<br/><br/>
Let’s see an example with `base=3`:
![14.png](images/14.png)
<br/>

<a id='cate10'></a>
### <font color='darkgreen'>10. HashingEncoder</font>
In <font color='blue'>**HashingEncoder**</font>, each original level is hashed, using some hashing algorithm, such as SHA-256. Then, the outcome is converted to integer and the module of that integer with respect to some (big) divisor is taken. By doing so, we have mapped each original string to an integer between 1 and divisor-1. Lastly, the integer that has been obtained by this procedure is one-hot encoded.
```python
def do_hash(string, output_dimension):
    hasher = hashlib.new('sha256')
    hasher.update(bytes(string, 'utf-8'))
    string_hashed = hasher.hexdigest()
    string_hashed_int = int(string_hashed, 16)
    string_hashed_int_remainder = string_hashed_int % output_dimension
    return string_hashed, string_hashed_int, string_hashed_int_remainder

hashing = x.apply(
    lambda string: pd.Series(
        do_hash(string, output_dimension), 
        index = ['x_hashed', 'x_hashed_int', 'x_hashed_int_remainder']
    )
)
hashing_encoding = hashing['x_hashed_int_remainder'].apply(
    lambda rem: pd.Series(np.diag(np.ones(output_dimension))[rem])
).astype(int)
```

**The fundamental property of hashing is that the resulting integer is uniformly distributed**. So, if you take a divisor big enough, it’s unlikely that two different strings are mapped to the same integer. Why would that be useful? Actually, this has a very practical application called [**“hashing trick”**](https://medium.com/value-stream-design/introducing-one-of-the-best-hacks-in-machine-learning-the-hashing-trick-bf6a9c8af18f).

<a id='cate11'></a>
### <font color='darkgreen'>11. TargetEncoder</font>
Suppose that you have two variables: one categorical (`x`) and one numeric (`y`). Say that you want to transform `x` into a numeric variable. **You may want to employ the information “carried” by `y`. An obvious idea is to take the mean of `y` for each level of `x`**. In formula:
> $$ enc_i = mean(y|x=i) $$

This is reasonable, but there’s a big problem with this approach: some groups may be too small or too diverse to be reliable. Many supervised encodings overcome this issue by choosing a middle way between the group mean and the global mean of `y`:
> $$ enc_i = w_i * mean(y|x-i) + (1-w_i) * mean(y) $$

where $w_i$ is between 0 and 1, depending on how “credible” is the group mean.

The next three algorithms (<font color='blue'>**TargetEncoder**</font>, <font color='blue'>[**MEstimateEncoder**](#cate12)</font> and <font color='blue'>[**JamesSteinEncoder**](#cate13)</font>) differ based on how they define $w_i$.

In <font color='blue'>**TargetEncoder**</font>, the weight depends on the group numerosity and on a parameter called “`smoothing`”. When smoothing is 0, we rely solely on group means. Then, as `smoothing` increases, the global mean weights more and more, leading to a stronger regularization.
```python
y_mean = y.mean()
y_level_mean = x.replace(y.groupby(x).mean())
weight = 1 / (1 + np.exp(-(count_encoding - 1) / smoothing))
target_encoding = y_level_mean * weight + y_mean * (1 - weight)
```
<br/><br/>
Let’s see how the outcome changes with some different values of smoothing:
![15.png](images/15.png)
<br/>

<a id='cate12'></a>
### <font color='darkgreen'>12. MEstimateEncoder</font>
<font color='blue'>**MEstimateEncoder**</font> resembles <font color='blue'>[**TargetEncoder**](#cate11)</font>, but $w_i$ depends on a parameter called “`m`”, which sets how much the global mean should weight in absolute terms. `m` is easy to understand because it can be considered as a number of observations: if the level has exactly `m` observations, then the level mean and the overall mean weight the same.
```python
y_mean = y.mean()
y_level_mean = x.replace(y.groupby(x).mean())
weight = count_encoding / (count_encoding + m)
m_estimate_encoding =  y_level_mean * weight + y_grand_mean * (1 - weight)
```
<br/><br/>
Let’s see how the outcome changes for different values of `m`:
![16.png](images/16.png)
<br/>

<a id='cate13'></a>
### <font color='darkgreen'>13. JamesSteinEncoder</font>
[**TargetEncoder**](#cate11) and [**MEstimateEncoder**](#cate12) depend both on the group numerosity and on the value of a parameter (<font color='brown'>respectively `smoothing` and `m`</font>) set by the user. This is not convenient, because setting these weights is a manual task.

**A natural question is the following: is there a way to set an optimal $w_i$, without the need of any human intervention?** The <font color='blue'>**JamesSteinEncoder**</font> tries to do so in a way that is statistically grounded.
```python
y_mean = y.mean()
y_var = y.var()
y_level_mean = x.replace(y.groupby(x).mean())
y_level_var = x.replace(y.groupby(x).var())

weight = 1 - (y_level_var / (y_var + y_level_var) * (len(set(x)) - 3) / (len(set(x)) - 1))
james_stein_encoding = y_level_mean * weight + y_mean * (1 - weight)
```
<br/><br/>
**The intuition is that the mean of a group with high variance should be trusted less.** Therefore, the higher the group variance, the lower the weight (<font color='brown'>if you want to know more about the formula, I suggest [this post](https://chris-said.io/2017/05/03/empirical-bayes-for-multiple-sample-sizes/) by Chris Said</font>).

Let’s see a numeric example:
![17.png](images/17.png)
<br/>

The <font color='blue'>**JamesSteinEncoder**</font> has two significant advantages: it provides better estimates than the maximum-likelihood estimator, and it doesn’t need any parameter setting.

<a id='cate14'></a>
### <font color='darkgreen'>14. GLMMEncoder</font>
<font color='blue'>**GLMMEncoder**</font> follows a totally different approach. Basically, it fits a Linear Mixed Effect Model on y. This approach exploits the fact that Linear Mixed Effect Models are designed precisely for handling homogeneous groups of observations (<font color='brown'>as well explained [here](https://stats.idre.ucla.edu/other/mult-pkg/introduction-to-linear-mixed-models/)</font>). Thus, the idea is to fit a model with no regressors (<font color='brown'>only the intercept</font>) and to use the levels as groups.

Then, the output is simply the sum of the intercept and the random effect of the group:
```
model = smf.mixedlm(formula = 'y ~ 1', data = y.to_frame(), groups = x).fit()
intercept = model.params['Intercept']
random_effect = x.replace({k: float(v) for k, v in model.random_effects.items()})
glmm_encoding = intercept + random_effect
```
<br/>

![18.png](images/18.png)
<br/>

<a id='cate15'></a>
### <font color='darkgreen'>15. WOEEncoder</font>
<font color='blue'>**WOEEncoder**</font> (<font color='brown'>which stands for “Weight of Evidence” Encoder</font>) can be employed only for binary target variables, i.e. target variables whose levels are 0/1. The idea behind Weight of Evidence is that you have two distributions:
* the distribution of 1s (`# of 1s in each group / # of 1s in all y`)
* the distribution of 0s (`# of 0s in each group / # of 0s in all y`)

**The heart of the algorithm is dividing the distribution of 1s by the distribution of 0s** (<font color='brown'>for each group</font>). Of course, the higher this value, the more confident we are that the group is “skewed” towards 1s, and viceversa. Then, the logarithm of this value is taken:
```python
y_level_ones = x.replace(y.groupby(x).apply(lambda l: (l == 1).sum()))
y_level_zeros = x.replace(y.groupby(x).apply(lambda l: (l == 0).sum()))
y_ones = (y == 1).sum()
y_zeros = (y == 0).sum()
nominator = y_level_ones / y_ones
denominator = y_level_zeros / y_zeros
woe_encoder = np.log(nominator / denominator)
```
<br/><br/>
As you can see, due to the presence of a logarithm in the formula, the output is not not directly interpretable. However, it works pretty well as a preprocessing step for machine learning.

<a id='cate16'></a>
### <font color='darkgreen'>16. LeaveOneOutEncoder</font>
All the 15 encoders seen up until now have a unique mapping.

However, if you plan to use the encoding as input for a predictive model (<font color='brown'>for example a gradient boosting</font>), this could be a problem. In fact, suppose that you use [**TargetEncoder**](#cate11). <font color='darkred'>**This would imply that you are introducing information about `y_train` inside `X_train`, which could lead to a serious risk of [overfitting](http://mlwiki.org/index.php/Overfitting).**</font>

**The point is: how to maintain a supervised encoding, while limiting the risk of overfitting?** <font color='blue'>**LeaveOneOutEncoder**</font> offers a brilliant solution. **It does a vanilla target encoding but, for each row, it does not consider the value of `y` observed for that row. In this way, it avoids [row-wise leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning)).**
```python
y_level_except_self = x.to_frame().apply(
  lambda row: y[x == row['x']].drop(row.name).to_list(), 
  axis = 1
)
leave_one_out_encoding = y_level_except_self.apply(np.mean)
```
<br/><br/>
![19.png](images/19.png)
<br/>

<a id='cate17'></a>
### <font color='darkgreen'>17. CatBoostEncoder</font>
[**CatBoost**](http://learningsys.org/nips17/assets/papers/paper_11.pdf) is a gradient boosting algorithm (<font color='brown'>like [**XGBoost**](https://xgboost.readthedocs.io/en/latest/python/python_intro.html) or LightGBM</font>) that has shown to work extremely well in a wide range of problems. The encoding algorithm is described in detail [here](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html) (<font color='brown'>our implementation is a little simplified, but it’s good for grasping the concept</font>)
> CatboostEncoder works basically like [**LeaveOneOutEncoder**](#cate16), but following an on-line approach.

But how to simulate an on-line behaviour in an off-line setting? Imagine that you have a table. Then, take a row somewhere in the middle of the table. What [**CatBoost**](http://learningsys.org/nips17/assets/papers/paper_11.pdf) does is pretending that the rows above the current row have been observed previously in time, while the rows below have yet to be observed (<font color='brown'>i.e. will be observed in the future</font>). Then, the algorithm does a leave-one-out encoding, but based only on the rows already observed.
![20.png](images/20.png)
<br/>
This may seem absurd. Why throwing away some information that could be useful? **You can see it simply as more extreme attempt of randomizing the output** (<font color='brown'>i.e. reducing overfitting</font>).